In [ ]:
"""
JUPYTER NOTEBOOK 1: EXERCISES

In this notebook, you'll explore:
1. Z-scores
2. Law of Large Numbers (LLN)
3. Central Limit Theorem (CLT)

We assume we have an industrial machine that produces rods of length ~ N(50, 2^2).
We'll walk through tasks to see how sample sizes affect our estimates.

You'll see hints about which methods to use, but you'll need to fill in the code.
"""

# =========================
# 1. SETUP
# =========================

# TODO: import the necessary libraries:
# HINTS:
#    - numpy for numerical operations
#    - matplotlib.pyplot for plotting
#    - from scipy.stats import norm (for normal distribution methods)
#    - optionally set a random seed for reproducibility

# <YOUR CODE HERE>


# -------------------------
# We'll define the true parameters of our "population."
# We have rods with mean length 50 cm and standard deviation 2 cm.
# true_mean = 50.0
# true_std = 2.0

# <YOUR CODE HERE: define true_mean and true_std>


In [ ]:

# =========================
# 2. Z-SCORES
# =========================

"""
TASK:
1. Simulate one rod length from our normal distribution (mean=50, std=2).
   Use: norm.rvs(loc=?, scale=?, size=?).

2. Compute its z-score. (z = (x - mean) / std).

3. Simulate many rod lengths (e.g., N=10_000) and compute their z-scores.
   - Plot a histogram of those z-scores and compare with the standard normal pdf.
     * HINT: use plt.hist(..., density=True) to get density.
     * HINT: norm.pdf(x_values, loc=0, scale=1) for standard normal PDF.
"""

# TODO: Step 1: simulate a single rod length
# <YOUR CODE HERE>

# TODO: Step 2: compute its z-score
# <YOUR CODE HERE>

# TODO: Step 3: simulate many rod lengths, compute z-scores, plot histogram vs. standard normal PDF
# <YOUR CODE HERE>



In [ ]:

# =========================
# 3. LAW OF LARGE NUMBERS
# =========================

"""
TASK:
1. Illustrate how the sample mean converges to the true mean as the sample size grows.
   - For sample_sizes = [10, 50, 100, 500, 1000, 5000, 10000], do:
     * Generate 'n' random samples from the normal distribution
     * Compute the mean
     * Store it
   - Print or plot those sample means to see if they approach 50.

2. Create a large sample (e.g., size=10_000) in one go, then compute the running mean 
   up to each index i (from 1 to 10,000). 
   - Plot the running mean and compare it to the true mean (50) with a horizontal line.
   - HINT: np.cumsum(...) gives a cumulative sum; to get the mean up to index i, 
     you can do cumulative_sum[i] / (i+1) or something similar.
"""

# TODO: Step 1: sample_sizes = [...]
# <YOUR CODE HERE>

# For each n in sample_sizes, generate normal samples, compute mean
# <YOUR CODE HERE>

# Print or plot the results
# <YOUR CODE HERE>

# TODO: Step 2: Single large sample. Compute running mean. Plot it against the true mean.
# <YOUR CODE HERE>

In [ ]:
# =========================
# 4. CENTRAL LIMIT THEOREM
# =========================

"""
TASK:
1. We'll pick various sample sizes, e.g. [1, 5, 30, 100, 1000].
2. For each n, repeatedly (e.g. 10_000 times) draw n samples from N(50, 2),
   compute the sample mean, store it in a list.
3. Plot a histogram of these sample means. Compare it to the normal distribution
   with mean=50, std=2/sqrt(n). 
   - HINT: norm.pdf(...) for the PDF, and pass loc=?, scale=?
"""

# TODO: define sample_sizes for the CLT demonstration
# <YOUR CODE HERE>

# for each n in sample_sizes:
#   - create a loop for num_experiments times
#   - generate sample of size n
#   - compute mean
#   - store
# - plot histogram and compare with theoretical PDF
# <YOUR CODE HERE>

In [ ]:
# =========================
# 5. PUTTING IT ALL TOGETHER
# =========================

"""
FINAL TASKS:
1. Create a single-run demonstration:
   - For sample_sizes = [10, 50, 100, 500, 1000, 5000]
     * For each n, generate a sample and compute its mean
     * Plot those means as a function of n, add a horizontal line at 50

2. Create a multiple-run demonstration:
   - For each n, do multiple runs (e.g. 1000) and plot the histogram of sample means 
     as we did in the CLT section, just more structured together.
"""

# TODO: implement the "single run" demonstration
# <YOUR CODE HERE>

# TODO: implement the "multiple runs" demonstration
# <YOUR CODE HERE>
